In [ ]:
# to enable table format
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# to disbale table format
# from google.colab import data_table
# data_table.disable_dataframe_formatter()

# !pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.5 MB/s eta 0:00:00


In [ ]:
# magic interactive shell command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# for data exploration
import pandas as pd
import numpy as np
import datetime as dt 

# remove warnings 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# for text processing 
import re
import nltk 

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import string 
string.punctuation

from bs4 import BeautifulSoup
# from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator

# gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex
from plotly.subplots import make_subplots
import pyLDAvis
# import pyLDAvis.gensim #dont skip this
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# to display all columns ana rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.min_rows',None)
pd.set_option('display.expand_frame_repr',True)

print('Libraries Imported!')



'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Libraries Imported!


In [ ]:
# upload file locally
# from google.colab import files
# uploaded = files.upload()

# file upload through google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # read raw data
# dataset = pd.read_csv(io.BytesIO(uploaded['sample_us.tsv'],sep='\t',error_bad_lines=False))
# df = pd.read_csv('/content/sample_us.tsv',sep='\t',error_bad_lines=False)
df = pd.read_csv("drive/MyDrive/datasets/watch_reviews.tsv",sep='\t',error_bad_lines=False)
df.head()
df.shape

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


(960204, 15)

In [ ]:
df[['product_title','review_body','product_category']].head()

,product_title,review_body,product_category
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Absolutely love this watch! Get compliments al...,Watches
1,Kenneth Cole New York Women's KC4944 Automatic...,I love this watch it keeps time wonderfully.,Watches
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,Scratches,Watches
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,"It works well on me. However, I found cheaper ...",Watches
4,Orient ER27009B Men's Symphony Automatic Stain...,Beautiful watch face. The band looks nice all...,Watches


In [ ]:
# check for time range of dataset
time_range=pd.Series(pd.to_datetime((df.review_date.sort_values(ascending=False)))).dropna()
time_diff=(time_range.iloc[0]-time_range.iloc[-1])
print(f'The dataset has a time period from {time_range.iloc[-1].year} to {time_range.iloc[0].year}, in total of {time_diff}.')

The dataset has a time period from 2001 to 2015, in total of 5261 days 00:00:00.


In [ ]:
# check dataset size and number of features
print(f"Dataset contians {df.shape[0]} rows and {df.shape[1]} columns")

Dataset contians 960204 rows and 15 columns


In [ ]:
# convert review body column to string datatype
df['review_body'] = df['review_body'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960204 non-null  object
 1   customer_id        960204 non-null  int64 
 2   review_id          960204 non-null  object
 3   product_id         960204 non-null  object
 4   product_parent     960204 non-null  int64 
 5   product_title      960202 non-null  object
 6   product_category   960204 non-null  object
 7   star_rating        960204 non-null  int64 
 8   helpful_votes      960204 non-null  int64 
 9   total_votes        960204 non-null  int64 
 10  vine               960204 non-null  object
 11  verified_purchase  960204 non-null  object
 12  review_headline    960197 non-null  object
 13  review_body        960204 non-null  object
 14  review_date        960200 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


**Building Up Text Corpus**

In [23]:
# filter() --> use to filter data from the dataset
# finding missing values
df['review_id'].isnull().sum()


0

Considering only those popular products which have more than 300 reviews 

In [28]:
# finding popular products using group by and filtering data where reviews > 300
popular_products = df.groupby(df['product_id']).filter(lambda x: x['review_id'].count()>=300)
print(f'Number of datapoints matching the criteria is: {len(popular_products)}')

# check null values for the new dataset
popular_products.isnull().sum()

Number of datapoints matching the criteria is: 132407


marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64

**Text Preprocessing**